In [1]:
import json
import os
from tqdm import tqdm
import os
import requests
import pickle
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# path of blocks
SOURCE_PATH = '../data/txn_data'
# SOURCE_PATH = 'testData'
# read files 
block_list = os.listdir(SOURCE_PATH)
sorted_block_list = [y for _, y in sorted([(int(a.split('.')[0]), a) for a in block_list])]
sorted_block_list = sorted_block_list
# print(sorted_block_list[:10])
print(f"There are {len(sorted_block_list)} blocks loaded")

There are 4087 blocks loaded


In [3]:
def get_block_txn_list(block_json_file: str) -> dict:
    """
    @input: a block json file, e.g., 1111.json
    @output: a list of transaction in that block
    """
    json_path = os.path.join(SOURCE_PATH, block_json_file)
    f = open(json_path)
    json_data = json.load(f)
    tx = json_data['tx']
    f.close()
    return tx

# Put transactions into 4 categories:
- Zero Ouputs
- One Output
- Two Outputs
- Multiple Outputs (three or more)

In [4]:
# CODES
PLACE_HOLDER_CODE = -1
ZERO_OUTPUT_CODE = 0
ONE_OUTPUT_CODE = 1
TWO_OUTPUT_CODE = 2
MULTIPLE_OUTPUT_CODE = 3


In [5]:
def output_count(txn: dict) -> int:
    """
    @input: a txn
    @output: number of "real" output in that transaction
    """
    output_list = txn['out']
    output_count = 0
    for output_addr in output_list:
        if 'addr' in output_addr:
            output_count += 1
    return output_count


In [6]:
transaction_address_summary = {}
two_output_address_summary = {}

def summarize_transaction_output():
    zeroOutput = []                      # store txn address with 0 output
    oneOutput = []                       # store txn address with 1 output
    twoOutput = []                       # store txn address with 2 output
    multipleOutput = []                  # store txn address with 3+ output

    global transaction_address_summary
    global two_output_address_summary
    
    for block_json_file in tqdm(sorted_block_list[:]):
        # get transaction list of that block
        txns_list = get_block_txn_list(block_json_file)
        for txn in txns_list:
            output_length = output_count(txn)
            if output_length == 0:
                zeroOutput.append(txn['hash'])
                transaction_address_summary[txn['hash']] = ZERO_OUTPUT_CODE
            elif output_length == 1:
                oneOutput.append(txn['hash'])
                transaction_address_summary[txn['hash']] = ONE_OUTPUT_CODE
            elif output_length == 2:
                twoOutput.append(txn['hash'])
                transaction_address_summary[txn['hash']] = TWO_OUTPUT_CODE
                two_output_address_summary[txn['hash']] = PLACE_HOLDER_CODE
            elif output_length >= 3:
                multipleOutput.append(txn['hash'])
                transaction_address_summary[txn['hash']] = MULTIPLE_OUTPUT_CODE

    print(f"zeoOutput: {len(zeroOutput)} | oneOutput: {len(oneOutput):,} \
| twoOutput: {len(twoOutput):,} | moreThanThreeOutput: {len(multipleOutput):,}")

summarize_transaction_output()

100%|██████████| 4087/4087 [05:11<00:00, 13.10it/s]


zeoOutput: 209 | oneOutput: 2,234,550 | twoOutput: 5,391,824 | moreThanThreeOutput: 767,973


In [7]:
# store two dictionaries

with open('../heuristic_data/transaction_address_summary.json', 'w') as f:
    json.dump(transaction_address_summary, f)

with open('../heuristic_data/two_output_address_summary.json', 'w') as f:
    json.dump(two_output_address_summary, f)